# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np 
import pandas as pd 
import requests 
from scipy import stats
import math


## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks
from sec import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN = 'sk_ab393a1caeb544bdaba4240a32a056c6'

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [5]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 56546004,
 'calculationPrice': 'close',
 'change': -3.14,
 'changePercent': -0.01725,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 178.87,
 'iexCloseTime': 1691438399809,
 'iexLastUpdated': 1691438399809,
 'iexMarketPercent': 19.190412402167368,
 'iexOpen': 182.17,
 'iexOpenTime': 1691415000134,
 'iexRealtimePrice': 178.87,
 'iexRealtimeSize': 4,
 'iexVolume': 2550022,
 'lastTradeTime': 1691438399999,
 'latestPrice': 178.85,
 'latestSource': 'Close',
 'latestTime': 'August 7, 2023',
 'latestUpdate': 1691438400511,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 279

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [6]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

30.06

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i + n]  

symbol_groups = list( chunks(stocks['Symbol'],100))
symbol_strings = []
my_columns = ['Ticker', 'Price', 'PE Ratio', 'Number of Shares to Buy']

for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns= my_columns)
print(symbol_strings)



['MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,ADP,AAP,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AMD,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ACGL,ANET,AJG,AIZ,T,ATO,ADSK,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BBWI,BAX,BDX,WRB,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,BG,CHRW,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC', 'CNP,CDAY,CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CTVA,CSGP,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DIS,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,LLY,EMR,ENPH,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FITB,FSLR', 'FE,FIS,FI,FLT,FMC,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GEHC,GEN,GNRC,GD,GE,GIS,GM,GPC,GILD,GL,GPN,GS,HAL,HIG,HAS,HCA,PEAK,HSIC,HSY,HES,HPE,HLT,HOLX,HD,HON,

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        final_dataframe.loc[len(final_dataframe)] = [symbol,data[symbol]['quote']['latestPrice'], data[symbol]['quote']['peRatio'], 'N/A'
                ]
        
        
final_dataframe
    

,Ticker,Price,PE Ratio,Number of Shares to Buy
0,MMM,105.23,-40.01,N/A
1,AOS,73.84,41.25,N/A
2,ABT,108.45,36.89,N/A
3,ABBV,150.33,35.46,N/A
4,ACN,315.87,28.18,N/A
...,...,...,...,...
498,YUM,135.11,31.72,N/A
499,ZBRA,248.23,19.87,N/A
500,ZBH,125.63,52.35,N/A
501,ZION,38.14,3.96,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [9]:
final_dataframe.sort_values('PE Ratio',inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [10]:
final_dataframe = final_dataframe[final_dataframe['PE Ratio']> 0]
final_dataframe.sort_values('PE Ratio')
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index',axis=1,inplace=True)
final_dataframe

,Ticker,Price,PE Ratio,Number of Shares to Buy
0,BRK.B,362.58,0.07,N/A
1,AAL,15.99,3.73,N/A
2,ZION,38.14,3.96,N/A
3,EMR,97.22,4.28,N/A
4,VLO,129.29,4.44,N/A
5,EQT,42.15,4.75,N/A
6,PSX,111.44,4.8,N/A
7,MPC,140.00,5.02,N/A
8,GM,37.09,5.17,N/A
9,CTRA,27.80,5.31,N/A


In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = (input("Enter the value of your portfolio:"))

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [12]:
portfolio_input()

Enter the value of your portfolio:100000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [13]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])
final_dataframe

,Ticker,Price,PE Ratio,Number of Shares to Buy
0,BRK.B,362.58,0.07,5
1,AAL,15.99,3.73,125
2,ZION,38.14,3.96,52
3,EMR,97.22,4.28,20
4,VLO,129.29,4.44,15
5,EQT,42.15,4.75,47
6,PSX,111.44,4.8,17
7,MPC,140.00,5.02,14
8,GM,37.09,5.17,53
9,CTRA,27.80,5.31,71


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [17]:
symbol = 'AAPL'
batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

pe_ratio = data[symbol]['quote']['peRatio']
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda


gross_profit = data['AAPL']['advanced-stats']['grossProfit']

ev_to_gross_profit = enterprise_value/gross_profit 



Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [20]:
rv_columns = [
    'Ticker', 'Price','Number of Shares To Buy', 'PE Ratio', 'PE Percentile', 'PB Ratio', 'PB Percentile', 'PS Ratio', 'PS Percentile', 'EV/EBITDA','EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile','RV Score'
    
]

rv_dataframe = pd.DataFrame(columns=rv_columns)
rv_dataframe

,Ticker,Price,Number of Shares To Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [23]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try: 
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.nan
        
        try: 
            ev_to_gross_profit = enterprise_value/gross_profit 
        except TypeError:
            ev_to_gross_profit = np.nan
        
        rv_dataframe.loc[len(rv_dataframe)] = [symbol,
                                               data[symbol]['quote']['latestPrice'], 
                                               'N/A',
                                               data[symbol]['quote']['peRatio'],
                                               'N/A',
                                               data[symbol]['advanced-stats']['priceToBook'], 
                                               'N/A',
                                               data[symbol]['advanced-stats']['priceToSales'],
                                               'N/A',
                                               ev_to_ebitda,
                                               'N/A',
                                               ev_to_gross_profit,
                                               'N/A',
                                               'N/A'
                                              ]
        
    

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
17,AFL,39.060,N/A,9.7,N/A,0.950934,N/A,NaN,N/A,NaN,N/A,N/A
18,AIG,31.950,N/A,-5.88,N/A,0.407454,N/A,NaN,N/A,NaN,N/A,N/A
20,AIZ,127.680,N/A,20.22,N/A,1.360264,N/A,NaN,N/A,NaN,N/A,N/A
26,ALL,97.880,N/A,7.1,N/A,1.173772,N/A,NaN,N/A,NaN,N/A,N/A
39,ANTM,293.560,N/A,12.24,N/A,2.304037,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,196.350,N/A,26.09,N/A,13.486066,N/A,17.533688,N/A,NaN,N/A,N/A
56,BAC,26.970,N/A,12.63,N/A,0.867662,N/A,NaN,N/A,NaN,N/A,N/A
64,BK,38.160,N/A,8.27,N/A,0.804550,N/A,NaN,N/A,NaN,N/A,N/A
65,BKNG,1866.170,N/A,30.78,N/A,12.571484,N/A,12.859665,N/A,NaN,N/A,N/A
75,C,55.140,N/A,9.52,N/A,0.576968,N/A,NaN,N/A,NaN,N/A,N/A


In [24]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares To Buy', 'PE Ratio',
       'PE Percentile', 'PB Ratio', 'PB Percentile', 'PS Ratio',
       'PS Percentile', 'EV/EBITDA', 'EV/EBITDA Percentile', 'EV/GP',
       'EV/GP Percentile', 'RV Score'],
      dtype='object')

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [26]:
for column in ['PE Ratio', 'PB Ratio', 'PS Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(),inplace=True)
    

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [27]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares To Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [32]:
metrics = { 'PE Ratio':'PE Percentile', 'PB Ratio': 'PB Percentile', 'PS Ratio':
       'PS Percentile', 'EV/EBITDA': 'EV/EBITDA Percentile', 'EV/GP':
       'EV/GP Percentile'}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row,metrics[metric]] = stats.percentileofscore(rv_dataframe[metric],rv_dataframe.loc[row,metric])
    
rv_dataframe

,Ticker,Price,Number of Shares To Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,105.23,N/A,-40.01,2.214651,7.45,77.512777,1.76,32.112436,-19.694851,0.851789,4.803688,29.642249,N/A
1,AOS,73.84,N/A,41.25,83.986371,4.87,64.310051,2.46,46.337308,11.284552,31.516184,6.369333,43.100511,N/A
2,ABT,108.45,N/A,36.89,78.705281,5.06,65.758092,4.68,71.379898,24.283494,85.689949,8.872942,63.543441,N/A
3,ABBV,150.33,N/A,35.46,77.172061,19.98,94.122658,4.67,70.783646,12.255693,37.819421,8.138417,57.410562,N/A
4,ACN,315.87,N/A,28.18,62.010221,8.28,81.09029,3.30,58.34753,16.876061,62.521295,9.823874,69.335605,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,YUM,135.11,N/A,31.72,72.061329,-4.31,5.792164,5.45,77.342419,21.088031,79.727428,9.480276,67.632027,N/A
583,ZBRA,248.23,N/A,19.87,44.633731,4.26,59.11414,2.32,44.122658,14.283802,48.892675,5.811684,37.819421,N/A
584,ZBH,125.63,N/A,52.35,90.630324,2.12,33.390119,3.65,61.839864,16.664085,61.158433,6.131293,41.226576,N/A
585,ZION,38.14,N/A,3.96,8.688245,1.17,13.287905,1.21,21.720613,3.031253,2.214651,1.262063,7.155026,N/A


In [38]:
metrics = { 'PE Ratio':'PE Percentile', 'PB Ratio': 'PB Percentile', 'PS Ratio':
       'PS Percentile', 'EV/EBITDA': 'EV/EBITDA Percentile', 'EV/GP':
       'EV/GP Percentile'}
metrics.values()

dict_values(['PE Percentile', 'PB Percentile', 'PS Percentile', 'EV/EBITDA Percentile', 'EV/GP Percentile'])

## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [40]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row,metrics[metric]]) 
    
    rv_dataframe.loc[row,'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares To Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,105.23,N/A,-40.01,2.214651,7.45,77.512777,1.76,32.112436,-19.694851,0.851789,4.803688,29.642249,28.46678
1,AOS,73.84,N/A,41.25,83.986371,4.87,64.310051,2.46,46.337308,11.284552,31.516184,6.369333,43.100511,53.850085
2,ABT,108.45,N/A,36.89,78.705281,5.06,65.758092,4.68,71.379898,24.283494,85.689949,8.872942,63.543441,73.015332
3,ABBV,150.33,N/A,35.46,77.172061,19.98,94.122658,4.67,70.783646,12.255693,37.819421,8.138417,57.410562,67.46167
4,ACN,315.87,N/A,28.18,62.010221,8.28,81.09029,3.30,58.34753,16.876061,62.521295,9.823874,69.335605,66.660988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,YUM,135.11,N/A,31.72,72.061329,-4.31,5.792164,5.45,77.342419,21.088031,79.727428,9.480276,67.632027,60.511073
583,ZBRA,248.23,N/A,19.87,44.633731,4.26,59.11414,2.32,44.122658,14.283802,48.892675,5.811684,37.819421,46.916525
584,ZBH,125.63,N/A,52.35,90.630324,2.12,33.390119,3.65,61.839864,16.664085,61.158433,6.131293,41.226576,57.649063
585,ZION,38.14,N/A,3.96,8.688245,1.17,13.287905,1.21,21.720613,3.031253,2.214651,1.262063,7.155026,10.613288


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [45]:
rv_dataframe.sort_values('RV Score',ascending=True,inplace = True)
rv_dataframe = rv_dataframe[0:50]
rv_dataframe.reset_index(drop=True,inplace=True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [46]:
portfolio_input()

Enter the value of your portfolio:2500000


In [47]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row,'Number of Shares To Buy'] = position_size/rv_dataframe.loc

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [48]:
rv_dataframe

,Ticker,Price,Number of Shares To Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,LNC,27.65,N/A,-1.56,7.666099,0.9705,9.199319,0.3051,3.407155,-1.600305,1.53322,0.305112,0.170358,4.39523
1,AAL,15.99,N/A,3.73,8.34753,-2.3800,6.473595,0.1975,1.192504,4.774829,7.155026,0.624499,2.214651,5.076661
2,AAL,15.99,N/A,3.73,8.34753,-2.3800,6.473595,0.1975,1.192504,4.774829,7.155026,0.624499,2.214651,5.076661
3,AAL,15.99,N/A,3.73,8.34753,-2.3800,6.473595,0.1975,1.192504,4.774829,7.155026,0.624499,2.214651,5.076661
4,GM,37.09,N/A,5.17,9.710392,0.7122,7.836457,0.3007,3.236797,1.489812,1.703578,1.069241,5.621806,5.621806
5,WBA,29.88,N/A,-7.78,5.962521,1.2200,13.969336,0.1895,0.851789,-7.235274,1.362862,1.359879,9.028961,6.235094
6,C,45.83,N/A,7.30,12.776831,0.4734,6.81431,0.6734,9.540034,4.016189,4.429302,0.710557,2.725724,7.25724
7,DXC,19.35,N/A,-6.81,6.303237,1.2100,13.798978,0.2802,2.555366,3.918655,3.747871,2.226906,14.991482,8.279387
8,SYF,34.75,N/A,6.42,10.902896,1.1500,12.86201,0.7154,10.221465,4.008789,4.258944,0.936875,4.770017,8.603066
9,WFC,45.06,N/A,11.27,19.591141,0.5110,7.325383,0.8144,11.584327,3.282806,2.555366,0.853825,3.747871,8.960818


## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: